In [1]:
import os
print os.getcwd()

/abaghel/projects/rnn-patient-sentiment-paper


In [2]:
import cPickle
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta
from keras.constraints import unitnorm, maxnorm
from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence

from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU

from sklearn.metrics import roc_auc_score

# This import to resolve some errors with tf version on office server
import tensorflow as tf
tf.python.control_flow_ops = tf

Using TensorFlow backend.


#  Simple Preprocessing without word2vec

In [68]:
import csv
import os
import numpy as np
import pandas as pd
import cPickle
from collections import defaultdict
import re


def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

     
def build_data_train_test(train_ratio = 0.8, clean_string=True):
    """
    Loads data and split into train and test sets.
    """
    revs = []
    vocab = defaultdict(float)
    # Pre-process train data set
    for i in xrange(len(text_train)):
        line = text_train[i]
        y = helpfull_train[i]
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': y, 
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': int(np.random.rand() < train_ratio)}
        revs.append(datum)
    # Pre-process test data set
    for i in xrange(len(text_test)):
        line = text_test[i]
        y = helpfull_test[i]
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': y, 
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': -1}
        revs.append(datum)      
    return revs, vocab


In [7]:
text=[]
helpfull=[]
knowledge=[]
staff=[]
ii=0
jj=0
files = [os.path.join("data/", f) for f in os.listdir('data/') if os.path.isfile(os.path.join("data/", f))]
for f in files:
    if f.endswith(".txt"):
        with open(f) as tsv:
            for line in csv.reader(tsv, dialect="excel-tab"):
                ii=ii+1
                if(len(line) > 6 and RepresentsInt(line[3]) and RepresentsInt(line[5]) and RepresentsInt(line[6]) ):
                    jj=jj+1
                    if int(line[5])>3:
                        text.append(line[2])
                        helpfull.append(int(1))
                    elif int(line[5])<3:
                        text.append(line[2])
                        helpfull.append(int(0))
                    #knowledge.append(int(line[6])-1)
                    #staff.append(int(line[3])-1)
    
        
        
print len(helpfull)
print len(text)
text_train=text[:34000]
text_test=text[34000:]
helpfull_train=helpfull[:34000]
helpfull_test=helpfull[34000:]

36683
36683


In [10]:
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype=np.float32)
    W[0] = np.zeros(k, dtype=np.float32)
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, 'rb') as f:
        header = f.readline()
        print header
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        print vocab_size*2/3
        for line in xrange(int(vocab_size*2.5/3)):
            #print line
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
                word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')  
            else:
                f.read(binary_len)
            if (line%100000==0):
                print line
    return word_vecs

def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25,0.25,k)  

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [69]:
revs, vocab = build_data_train_test(train_ratio=0.8, clean_string=True)
max_l = np.max(pd.DataFrame(revs)['num_words'])
print 'data loaded!'
print 'number of sentences: ' + str(len(revs))
print 'vocab size: ' + str(len(vocab))
print 'max sentence length: ' + str(max_l)

data loaded!
number of sentences: 36683
vocab size: 35971
max sentence length: 247


In [12]:
w2v = {}

In [13]:
add_unknown_words(w2v, vocab)
W, word_idx_map = get_W(w2v)

In [71]:
print (len(W))
print (len(word_idx_map.items()))
for i in range(10):
    print word_idx_map.keys()[i]

35972
35971
blgh
pasnecker
colonoscopy
sonja
gag
woods
hanging
woody
localized
dianostic


# Prepare for training

In [72]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
#     while len(x) < max_l:
#         x.append(0)
    return x

def make_idx_data(revs, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, val, test = [], [], []
    y_train, y_val, y_test = [], [] , []
    for rev in revs:
        sent = get_idx_from_sent(rev['text'], word_idx_map, max_l, kernel_size)
#         sent.append(rev['y'])
        if rev['split'] == 1:
            train.append(sent)
            y_train.append(int(rev["y"]))
        elif rev['split'] == 0:
            val.append(sent)
            y_val.append(int(rev["y"]))
        else:
            test.append(sent)
            y_test.append(int(rev["y"]))
            
            
#     train = np.array(train, dtype=np.int)
#     val = np.array(val, dtype=np.int)
#     test = np.array(test, dtype=np.int)
    train = sequence.pad_sequences(train, maxlen=max_l)
    y_train = np.array(y_train, dtype=np.int)
    
    val = sequence.pad_sequences(val, maxlen=max_l)
    y_val = np.array(y_val, dtype=np.int)
    
    test = sequence.pad_sequences(test, maxlen=max_l)
    y_test = np.array(y_test, dtype=np.int)
    
    return [train, val, test, y_train, y_val, y_test]

In [73]:
datasets = make_idx_data(revs, word_idx_map, max_l=200,kernel_size=5)

In [74]:
train, y_train = datasets[0], datasets[3]
val, y_val = datasets[1], datasets[4]
test, y_test = datasets[2], datasets[5]

In [75]:
print len(train), train.shape, len(y_train), y_train.shape, y_train.mean()
print np.unique(y_train)

27179 (27179, 200) 27179 (27179,) 0.686743441628
[0 1]


In [76]:
print len(val), val.shape, len(y_val), y_val.shape, y_val.mean()
print np.unique(y_val)

6821 (6821, 200) 6821 (6821,) 0.69154082979
[0 1]


In [77]:
print len(test), test.shape, len(y_test), y_test.shape, y_test.mean()
print np.unique(y_test)

2683 (2683, 200) 2683 (2683,) 0.699217294074
[0 1]


# Try the model same as IMDB

In [66]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(vocab)+1, 128, dropout=0.2, mask_zero=True))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [67]:
print('Train...')
model.fit(train, y_train, batch_size=50, nb_epoch=15,validation_data=(val, y_val))
# score, acc = model.evaluate(test, y_test,
#                             batch_size=50)
# print('Test score:', score)
# print('Test accuracy:', acc)

Train...
Train on 27234 samples, validate on 6766 samples
Epoch 1/15
27234/27234 [==============================] - 435s - loss: 0.3745 - acc: 0.8288 - val_loss: 0.1986 - val_acc: 0.9138
Epoch 2/15
27234/27234 [==============================] - 432s - loss: 0.2382 - acc: 0.8960 - val_loss: 0.1803 - val_acc: 0.9228
Epoch 3/15
27234/27234 [==============================] - 430s - loss: 0.1947 - acc: 0.9166 - val_loss: 0.1784 - val_acc: 0.9261
Epoch 4/15
27234/27234 [==============================] - 431s - loss: 0.1652 - acc: 0.9286 - val_loss: 0.1705 - val_acc: 0.9291
Epoch 5/15
27234/27234 [==============================] - 432s - loss: 0.1459 - acc: 0.9347 - val_loss: 0.1644 - val_acc: 0.9307
Epoch 6/15
27234/27234 [==============================] - 427s - loss: 0.1250 - acc: 0.9439 - val_loss: 0.1808 - val_acc: 0.9289
Epoch 7/15
27234/27234 [==============================] - 434s - loss: 0.1227 - acc: 0.9442 - val_loss: 0.2058 - val_acc: 0.9268
Epoch 8/15
27234/27234 [===============

In [78]:
score, acc = model.evaluate(test, y_test,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)

2683/2683 [==============================] - 12s    
('Test score:', 0.20978614839125026)
('Test accuracy:', 0.92396571182555953)
